# Import Packages

In [2]:
import numpy as np
import pandas as pd

# Numpy Methods

## `np.linalg.solve(A, B)`

In [3]:
# AX = B
A = np.array([[1, 1, 1, 1],
              [2, 3, 0, -1],
              [-3, 4, 1, 2],
              [1, 2, -1, 1]], dtype="float32")

B = np.array([13, 1, 10, 1], dtype="float32")

X = np.linalg.solve(A, B)
print(X)

[2.1111112  0.37037036 6.185185   4.3333335 ]


## `np.linalg.inv(A)`

In [4]:
A = np.array([[3,  1, -1],
              [4,  7, -3],
              [2, -2,  5]], dtype="float32")

A_invers = np.linalg.inv(A)
print(A_invers)

[[ 0.3493976  -0.03614458  0.04819277]
 [-0.31325302  0.20481928  0.06024097]
 [-0.26506025  0.09638554  0.20481928]]


# Metode dari Materi

## Eliminasi Gauss

In [5]:
def Gauss(A, B, printStep=True):
    size = B.size
    matrix = np.concatenate((A, B.reshape(size, 1)), axis=1)
    
    for i in range(1, size):
        if printStep:
            print("step", i, "\n")
            
        rowBefore = matrix[i-1]/matrix[i-1, i-1] # depannya 1

        for j in range(i, size):
            matrix[j] -= (rowBefore*matrix[j, i-1])
            if printStep:
                print(matrix, "\n")
    
    return matrix

## Eliminasi Gauss-Jordan

### Dari Awal

In [6]:
def GaussJordan(A, B, printStep=False, showInv=False, showB=True):
    size = B.size
    matrix = A
    if showB:
        matrix = np.concatenate((A, B.reshape(size, 1)), axis=1)
    if showInv:
        identity = np.identity(size, dtype="float32")
        matrix = np.concatenate((matrix, identity), axis=1)
    
    for i in range(0, size):
        if printStep:
            print("step", i+1, "\n")
            
        matrix[i] /= matrix[i, i]

        for j in range(0, size):
            if i == j:
                continue
                
            matrix[j] -= matrix[i] * matrix[j, i]
            if printStep:
                print(matrix, "\n")
        
    return matrix

### Lanjutan Eliminasi Gauss

In [7]:
def GaussJordan2(A, B, printStep=False, showInv=False, showB=True):
    size = B.size
    matrix = A
    if showB:
        matrix = np.concatenate((A, B.reshape(size, 1)), axis=1)
    if showInv:
        identity = np.identity(size, dtype="float32")
        matrix = np.concatenate((matrix, identity), axis=1)

    for i in range(1, size):
        if printStep:
            print("step", i, "\n")
            
        rowBefore = matrix[i-1]/matrix[i-1, i-1]  # depannya 1

        for j in range(i, size):
            matrix[j] -= (rowBefore*matrix[j, i-1])
            if printStep:
                print(matrix, "\n")
    
    for i in range(size-1, -1, -1):
        if printStep:
            print("step backward", i, "\n")
            
        matrix[i] /= matrix[i, i]
        for j in range(size-1, -1, -1):
            if i == j:
                continue
            matrix[j] -= matrix[i]*matrix[j, i]
            if printStep:
                print(matrix, "\n")
                
            
    return matrix

## Iterasi - Jacobi

In [8]:
# AX = B
def Jacobi(A, B, X_before, es=0.005, iter=1, printStep=False):
    size = B.size
    X = np.zeros(size)
    RAE = np.zeros(size)

    # iterasi X -> x1, x2, x3, dst... || x, y, z, dst...
    for i in range(0, size):
        X[i] = B[i]
        for j in range(0, size):
            if i != j:
                X[i] -= A[i, j]*X_before[j]
        X[i] /= A[i, i]

    for i in range(0, size):
        RAE[i] = abs((X[i] - X_before[i])/X[i])
        
    if printStep:
        print("\niter", iter)
        print("X   ", X)
        print("RAE ", RAE)

    if np.all(RAE < es) or iter > 100:
        return X

    return Jacobi(A, B, X, es, iter+1, printStep)

## Iterasi - Gauss-Seidel

In [9]:
# AX = B
def GaussSeidel(A, B, X_before, es=0.005, iter=1, printStep=False):
    size = B.size
    X = np.zeros(size)
    RAE = np.zeros(size)
    X_new = X_before.copy()                # bedanya ini

    # iterasi X -> x1, x2, x3, dst... || x, y, z, dst...
    for i in range(0, size):
        X[i] = B[i]
        for j in range(0, size):
            if i != j:
                X[i] -= A[i, j]*X_new[j]   # ini
        X[i] /= A[i, i]
        X_new[i] = X[i]                    # dan ini

    for i in range(0, size):
        RAE[i] = abs((X[i] - X_before[i])/X[i])

    if printStep:
        print("\niter", iter)
        print("X   ", X)
        print("RAE ", RAE)

    if np.all(RAE < es) or iter > 100:
        return X

    return GaussSeidel(A, B, X, es, iter+1, printStep)

# Driver Code

- **Gauss**
  ```py
  result = Gauss(A, B, printStep=True)
  ```

- **Gauss Jordan (dari awal)** | urutan: A B A_inv | result bukan untuk invers
  ```py
  resultMatrix = GaussJordan(A, B, printStep=True, showInv=False, showB=True)
  result = resultMatrix[:, -1]
  ```

- **Gauss Jordan (lanjutan gauss)** | urutan A B A_inv | result bukan untuk invers
  ```py
  resultMatrix = GaussJordan2(A, B, printStep=True, showInv=False, showB=True)
  result = resultMatrix[:, -1]
  ```
  
- **Jacobi**
  ```py
  # X0 => terkaan awal
  X0 = np.array([0, 0, 0], dtype="float32")
  result = Jacobi(A, B, X0, es=0.005, printStep=True)
  ```
  
- **Gauss Seidel**
  ```py
  # X0 => terkaan awal
  X0 = np.array([0, 0, 0], dtype="float32")
  result = GaussSeidel(A, B, X0, es=0.005, printStep=True)
  ```

In [16]:
A = np.array([[8, -2, -1, 4],
             [2, 8, 2, -1],
             [-1, 2, 8, 2],
             [4, -1, 2, 8]], dtype="float32")

B = np.array([17, 9, -10, 4], dtype="float32")

# copy here
# X0 => terkaan awal
X0 = np.array([0, 0, 0, 0], dtype="float32")
result = GaussSeidel(A, B, X0, es=0.05, printStep=True)
# end of copy here

# print(np.linalg.solve(A, B))

print("\nresult", result, sep="\n")


iter 1
X    [ 2.125       0.59375    -1.1328125  -0.20507812]
RAE  [1. 1. 1. 1.]

iter 2
X    [ 2.234375    0.82397461 -1.12542725 -0.23283386]
RAE  [0.04895105 0.27940741 0.00656218 0.11920834]

iter 3
X    [ 2.30673218  0.80056953 -1.1035924  -0.2773968 ]
RAE  [0.03136783 0.02923553 0.01978525 0.1606469 ]

iter 4
X    [ 2.32589173  0.78475057 -1.08610198 -0.29332655]
RAE  [0.00823751 0.02015796 0.01610385 0.05430723]

iter 5
X    [ 2.33208817  0.77683763 -1.07936675 -0.29909769]
RAE  [0.00265703 0.01018608 0.00623998 0.01929517]

result
[ 2.33208817  0.77683763 -1.07936675 -0.29909769]
